# From SQL to pandas challenge 11

In [ ]:
# import libraries
import pandas as pd

# load data
# This code is made to load our data stored on Google Drive
def gd_path(file_id):
    """Generate a shareable link from Google Drive file id."""
    return f"https://drive.google.com/uc?export=download&id={file_id}"

# Google Drive file ids
files_id = {
    "titleauthor": "1F1JOiYXStWacOBca6coNVfyVtoST7ZgD",
    "titles": "1PLdn50N9GRa53ZbuVWo0l47F_IXdvlEm",
    "sales": "1fzFc9rwYmVIPaGOFmhLVxCi3kg19vNU2", 
    "roysched": "1zPRZPoFPEMKyrNR5VSENeYFHGCBZmxbs", 
    "publishers": "1s9E8_AVOziTrowb3wyh2jg3PV763VOyq",
    "employee": "1h9mUjsVqpP74b1w0x7KOw37n_n9Ulkt5", 
    "authors": "1fEF89Nhe61EebAljKlwFwfEuokK0o6aJ"
}

# Read data from Google Drive
sales = pd.read_csv(gd_path(files_id["sales"]), sep=";")
titles = pd.read_csv(gd_path(files_id["titles"]), sep=";")
publishers = pd.read_csv(gd_path(files_id["publishers"]), sep=";")
employee = pd.read_csv(gd_path(files_id["employee"]), sep=";")
authors = pd.read_csv(gd_path(files_id["authors"]), sep=";")
titleauthor = pd.read_csv(gd_path(files_id["titleauthor"]), sep=";")
roysched = pd.read_csv(gd_path(files_id["roysched"]), sep=";")

## 1. Using LEFT JOIN: in which cities has "Is Anger the Enemy?" been sold?

In [ ]:
sales.columns

Index(['stor_id', 'ord_num', 'ord_date', 'qty', 'payterms', 'title_id'], dtype='object')

In [ ]:
publishers.columns

Index(['pub_id', 'pub_name', 'city', 'state', 'country'], dtype='object')

In [ ]:
titles.columns

Index(['title_id', 'title', 'type', 'pub_id', 'price', 'advance', 'royalty',
       'ytd_sales', 'notes', 'pubdate'],
      dtype='object')

In [ ]:
titleauthor.columns

Index(['au_id', 'title_id', 'au_ord', 'royaltyper'], dtype='object')

In [ ]:
titles[titles.title.str.contains("Anger")]

,title_id,title,type,pub_id,price,advance,royalty,ytd_sales,notes,pubdate
11,PS2091,Is Anger the Enemy?,psychology,736,10.95,2275.0,12,2045,Carefully researched study of the effects of s...,1991-06-15 00:00:00


In [ ]:
titles.merge(
    publishers,
    how = "inner",
    left_on ="pub_id",
    right_on = "pub_id"
).title[7]

'Is Anger the Enemy?'

In [ ]:
titles_nem = titles.merge(
    publishers,
    how = "inner",
    left_on ="pub_id",
    right_on = "pub_id"
)

titles_nem[titles_nem.title.str.contains('Anger')][['title','city']]

,title,city
7,Is Anger the Enemy?,Boston


### Hint:

In SQL the syntax is:

```sql
SELECT p.city
FROM publishers AS p
LEFT JOIN titles AS t
ON p.pub_id = t.pub_id
WHERE t.title = 'Is Anger the Enemy?';
```

## 2. Select all the book titles that have a link to the employee Howard Snyder 
    (he works for the publisher that has published those books).

In [ ]:
big_q_emp = employee.merge(
    titles,
    how = "inner",
    left_on = "pub_id",
    right_on = "pub_id"
).merge(
    publishers,
    how="inner",
    left_on = "pub_id",
    right_on = "pub_id"
)

big_q_emp[(big_q_emp.fname== "Howard") & (big_q_emp.lname == "Snyder")][["title", "fname","lname"]]

,title,fname,lname
135,You Can Combat Computer Stress!,Howard,Snyder
136,Is Anger the Enemy?,Howard,Snyder
137,Life Without Fear,Howard,Snyder
138,Prolonged Data Deprivation: Four Case Studies,Howard,Snyder
139,Emotional Security: A New Algorithm,Howard,Snyder


### Hint:

In SQL the syntax is:

```sql
SELECT t.title
FROM employee e
JOIN titles t
ON e.pub_id = t.pub_id
WHERE e.fname = 'Howard'
AND e.lname = 'Snyder';
```

## 3. Using the `merge` of your choice: Select the book title with highest number of sales (qty)

In [ ]:
titles.columns

Index(['title_id', 'title', 'type', 'pub_id', 'price', 'advance', 'royalty',
       'ytd_sales', 'notes', 'pubdate'],
      dtype='object')

In [ ]:
sales.columns

Index(['stor_id', 'ord_num', 'ord_date', 'qty', 'payterms', 'title_id'], dtype='object')

In [ ]:
book_qunt = titles.merge(
    sales,
    how = "inner",
    left_on = "title_id",
    right_on = "title_id"
).groupby("title").qty.sum()

book_qunt.sort_values(ascending=False).head(1)

title
Is Anger the Enemy?    108
Name: qty, dtype: int64

### Hint:

In SQL the syntax is:

```sql
SELECT t.title, SUM(qty)
FROM sales AS s 
JOIN titles t
ON s.title_id = t.title_id
GROUP BY t.title_id
ORDER BY SUM(qty) desc
LIMIT 1;
```

# 4. Select all book titles and the full name of their author(s).
      
      - If a book has multiple authors, all authors must be displayed (in 
      multiple rows).
      
      - Books with no authors and authors with no books should not be displayed.

In [ ]:
titles.title_id.unique()

array(['BU1032', 'BU1111', 'BU2075', 'BU7832', 'MC2222', 'MC3021',
       'MC3026', 'PC1035', 'PC8888', 'PC9999', 'PS1372', 'PS2091',
       'PS2106', 'PS3333', 'PS7777', 'TC3218', 'TC4203', 'TC7777'],
      dtype=object)

In [ ]:
all_title = titleauthor.merge(
    titles,
    how ="inner",
    left_on ="title_id",
    right_on ="title_id"
).merge(
    authors,
    how="inner",
    left_on ="au_id",
    right_on = "au_id"
)

#all_title.columns
all_title[["au_fname", "au_lname","title"]].sort_values(by="title")


,au_fname,au_lname,title
4,Cheryl,Carson,But Is It User Friendly?
21,Livia,Karsen,Computer Phobic AND Non-Phobic Individuals: Be...
8,Stearns,MacFeather,Computer Phobic AND Non-Phobic Individuals: Be...
5,Michael,O'Leary,Cooking with Computers: Surreptitious Balance ...
7,Stearns,MacFeather,Cooking with Computers: Surreptitious Balance ...
15,Charlene,Locksley,Emotional Security: A New Algorithm
16,Reginald,Blotchet-Halls,Fifty Years in Buckingham Palace Kitchens
20,Anne,Ringer,Is Anger the Enemy?
23,Albert,Ringer,Is Anger the Enemy?
24,Albert,Ringer,Life Without Fear


### Hint:

In SQL the syntax is:

```sql
SELECT
    t.title,
    a.au_fname,
    a.au_lname
FROM titles t
INNER JOIN titleauthor ta 
ON t.title_id = ta.title_id
INNER JOIN authors a 
ON ta.au_id = a.au_id;
```

## 5. Select the full name of authors of Psychology books

   Bonus hint: if you want to prevent duplicates but allow authors with shared
   last names to be displayed, you can concatenate the first and last names
   with CONCAT(), and use the DISTINCT clause on the concatenated names.

In [ ]:
all_title[all_title.type == "psychology"][["au_lname","au_fname"]].drop_duplicates()

,au_lname,au_fname
0,White,Johnson
8,MacFeather,Stearns
15,Locksley,Charlene
20,Ringer,Anne
21,Karsen,Livia
23,Ringer,Albert


### Hint:

In SQL the syntax is:

```sql
SELECT DISTINCT CONCAT(a.au_fname, " ", a.au_lname) AS full_name
FROM authors a
INNER JOIN titleauthor ta ON a.au_id = ta.au_id
INNER JOIN titles t ON ta.title_id = t.title_id
WHERE t.type = "Psychology";
```

## 6. Explore the table roysched and try to grasp the meaning of each column. 
   The notes below will help:
   
   - "Royalty" means the percentage of the sale price paid to the author(s).
   
   - Sometimes, the royalty may be smaller for the first few sales (which have
     to cover the publishing costs to the publisher) but higher for the sales 
     above a certain threshold.
     
   - In the "roysched" table each title_id can appear multiple times, with
     different royalty values for each range of sales.
     
   - Select all rows for particular title_id, for example "BU1111", and explore
	 the data.

In [ ]:
roysched.sort_values(by=["title_id", "lorange"]).head(10)

,title_id,lorange,hirange,royalty
0,BU1032,0,5000,10
1,BU1032,5001,50000,12
49,BU1111,0,4000,10
50,BU1111,4001,8000,12
51,BU1111,8001,10000,14
52,BU1111,12001,16000,16
53,BU1111,16001,20000,18
54,BU1111,20001,24000,20
55,BU1111,24001,28000,22
56,BU1111,28001,50000,24


In [ ]:
roysched[roysched.title_id == "BU1111"]

,title_id,lorange,hirange,royalty
49,BU1111,0,4000,10
50,BU1111,4001,8000,12
51,BU1111,8001,10000,14
52,BU1111,12001,16000,16
53,BU1111,16001,20000,18
54,BU1111,20001,24000,20
55,BU1111,24001,28000,22
56,BU1111,28001,50000,24


### Hint:

In SQL the syntax is:

```sql
SELECT * FROM roysched WHERE title_id = "BU1111";
```

## 7. Select all the book titles and the maximum royalty they can reach.
    Display only titles that are present in the roysched table.

In [ ]:
roysched.groupby("title_id").royalty.max()

title_id
BU1032    12
BU1111    24
BU2075    24
BU7832    24
MC2222    20
MC3021    24
PC1035    18
PC8888    16
PS1372    18
PS2091    16
PS2106    16
PS3333    16
PS7777    12
TC3218    24
TC4203    22
TC7777    14
Name: royalty, dtype: int64

In [ ]:
roysched.columns

Index(['title_id', 'lorange', 'hirange', 'royalty'], dtype='object')

In [ ]:
roysched[["title_id", "royalty"]].sort_values(by="title_id")

,title_id,royalty
0,BU1032,10
1,BU1032,12
51,BU1111,14
52,BU1111,16
53,BU1111,18
...,...,...
66,TC4203,10
72,TC4203,22
64,TC7777,12
63,TC7777,10


In [ ]:
titles[["title_id", "royalty"]].sort_values(by="title_id")

,title_id,royalty
0,BU1032,10
1,BU1111,10
2,BU2075,24
3,BU7832,10
4,MC2222,12
5,MC3021,24
6,MC3026,0
7,PC1035,16
8,PC8888,10
9,PC9999,0


In [ ]:
titles.columns

Index(['title_id', 'title', 'type', 'pub_id', 'price', 'advance', 'royalty',
       'ytd_sales', 'notes', 'pubdate'],
      dtype='object')

In [ ]:
new_au_roy= roysched.merge(
    titles,
    how ="innler",
    left_on = "title_id",
    right_on = "title_id"
)

In [ ]:
new_au_roy.columns
new_au_roy.sample(4)
new_au_roy.groupby("title").royalty_y.max()

title
But Is It User Friendly?                                           16
Computer Phobic AND Non-Phobic Individuals: Behavior Variations    10
Cooking with Computers: Surreptitious Balance Sheets               10
Emotional Security: A New Algorithm                                10
Fifty Years in Buckingham Palace Kitchens                          14
Is Anger the Enemy?                                                12
Life Without Fear                                                  10
Onions, Leeks, and Garlic: Cooking Secrets of the Mediterranean    10
Prolonged Data Deprivation: Four Case Studies                      10
Secrets of Silicon Valley                                          10
Silicon Valley Gastronomic Treats                                  12
Straight Talk About Computers                                      10
Sushi, Anyone?                                                     10
The Busy Executive's Database Guide                                10
The Gourmet Mi

In [ ]:
new_au_roy.columns

Index(['title_id', 'lorange', 'hirange', 'royalty_x', 'title', 'type',
       'pub_id', 'price', 'advance', 'royalty_y', 'ytd_sales', 'notes',
       'pubdate'],
      dtype='object')

### Hint:

In SQL the syntax is:

```sql
SELECT t.title, MAX(r.royalty) max_royalty
FROM titles t
INNER JOIN roysched r 
ON t.title_id = r.title_id
GROUP BY t.title
ORDER BY max_royalty DESC;
```